In [8]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
df_processed = pd.read_csv('../DATASET/DATA[T].csv')

df_original = pd.read_csv('../DATASET/DATA.csv')

In [10]:
X_features = df_processed.drop('Churn', axis=1)
similarity_matrix = cosine_similarity(X_features)
similarity_df = pd.DataFrame(similarity_matrix, index=df_original['customerID'], columns=df_original['customerID'])

In [11]:
def get_service_recommendations(customer_id, num_recommendations=3):
    """
    Generates service recommendations for a given customer based on finding similar users.
    """
    # 1. Find the top 10 most similar customers (excluding the customer themselves)
    similar_customers = similarity_df[customer_id].sort_values(ascending=False).iloc[1:11]

    # 2. Get the service details for these similar customers from the ORIGINAL dataframe
    similar_customer_services = df_original[df_original['customerID'].isin(similar_customers.index)]

    # 3. Get the services of our target customer from the ORIGINAL dataframe
    target_customer_services = df_original[df_original['customerID'] == customer_id]

    # 4. Identify services that the target customer doesn't have but similar customers do
    recommendations = {}
    # List of services we can potentially upsell
    service_columns = ['PhoneService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

    for service in service_columns:
        # Check if the target customer does NOT have the service
        if target_customer_services[service].iloc[0] == 'No':
            # Count how many of the similar customers DO have that service
            recommendation_score = similar_customer_services[service].value_counts().get('Yes', 0)
            if recommendation_score > 0:
                recommendations[service] = recommendation_score

    # 5. Sort the potential recommendations by how many similar users have them
    sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)

    # Return the top N recommendations
    return dict(sorted_recommendations[:num_recommendations])

In [ ]:
sample_customer_id = '8091-TTVAX'

recommended_services = get_service_recommendations(sample_customer_id)

print("\n" + "="*50)
if recommended_services:
    print(f"TOP RECOMMENDATIONS FOR CUSTOMER {sample_customer_id}:")
    for service, score in recommended_services.items():
        print(f"- Recommend '{service}' (Subscribed to by {score} out of 10 similar customers)")
else:
    print(f"No new service recommendations for customer {sample_customer_id}.")
print("="*50)


TOP RECOMMENDATIONS FOR CUSTOMER 8091-TTVAX:
- Recommend 'OnlineBackup' (Subscribed to by 4 out of 10 similar customers)
- Recommend 'OnlineSecurity' (Subscribed to by 1 out of 10 similar customers)
